# Week 06. Regression

과제는 총 3개의 cell과 markdown 하나를 채워주셔야합니다

1~3 문제는 영상과 실습자료에 나와있는 것들을 적절히 응용하시면 됩니다

마지막 4번의 경우 본인의 생각을 마크다운으로 작성하는 부분입니다

주석의 경우 이미지, 테이블 등의 표현이 어려운 관계로 받지 않겠습니다

### 실습 00. Load dataset

이번 과제에 사용할 dataset은 parkinson UPDRS dataset입니다

ML dataset repository인 UCI에서 받을 수 있습니다

실습시간과 같은 데이터셋이지만 다른 타깃을 사용합니다

6개의 파일중 y2를 불러와주세요

In [36]:
import pandas as pds
import numpy as np
import os

from plotly.subplots import make_subplots
from plotly import graph_objs as go

x_train=pds.read_csv('07_reg_x_train.csv')
x_test=pds.read_csv('07_reg_x_test.csv')
y_train=pds.read_csv('07_reg_y2_train.csv')
y_test=pds.read_csv('07_reg_y2_test.csv')

### 실습 01. Least sqaures methods

Least squares methods를 실습하겠습니다

실습영상에는 scaling을 적용하지 않았습니다

이번에는 scaling을 적용하시고 coefficients $\beta$를 구해주세요

그리고 decision tree regression도 수행하신 후 feature importance를 확인해주세요

$\beta$와 feature importance에 대해 간단히 비교하고, scaling된 변수를 잘 반영하고 있는지 정리해주세요

In [45]:
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn import tree

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)
x_train = pds.DataFrame(x_train)
x_test = pds.DataFrame(x_test)

x_train['bias'], x_test['bias'] = 1,1
cols = x_train.columns
xt_dot_x=np.dot(x_train.values.transpose(), x_train.values)+1e-15

inv_xdots=np.linalg.inv(xt_dot_x)

invx_dot_xt=np.dot(inv_xdots, x_train.values.transpose())

beta=np.dot(invx_dot_xt, y_train.values.ravel())

pds.Series(beta, index=cols)

0         3.253712
1         2.840203
2        -2.296877
3         0.868176
4        -1.421820
5        -1.591560
6       -13.737881
7        -0.423559
8        17.161144
9         0.423944
10        0.122811
11     -282.889347
12        0.889446
13        0.103178
14      280.672483
15       -1.506014
16       -1.969384
17        0.212005
18       -2.611495
19        1.479198
20        0.421637
bias     29.047096
dtype: float64

In [48]:
print(f'r2 score: {r2_score(y_test.values.ravel(),(x_test@beta).values.ravel()):4}')
print(f'RMSE: {mean_squared_error(y_test.values.ravel(),(x_test@beta).values.ravel())**0.5:.4}')
print(f'MAE: {mean_absolute_error(y_test.values.ravel(),(x_test@beta).values.ravel()):4}')

r2 score: 0.24220489986574623
RMSE: 9.12
MAE: 7.391319784588579


In [78]:
DTreg = tree.DecisionTreeRegressor()
DTreg.fit(x_train, y_train)

predict = DTreg.predict(x_test)
predict = pds.DataFrame(predict)
score = DTreg.score(x_test, y_test)


for i in range(0,20):
    print(DTreg.feature_importances_[i])

0.5227212296032987
0.38569312531375005
0.003678235211896721
0.058132203503781274
2.4493001554696466e-06
4.703158861730877e-06
1.3598144468521599e-05
0.0003363698155999231
1.5764086788966947e-05
0.00037563728135823614
3.239858948138955e-06
1.8238254469484365e-06
0.0008533255394405829
0.00031212227501117997
0.0029531604248234093
8.596045129935434e-07
2.4374323136835724e-05
0.022928403901285532
0.0019105715709553726
3.8803256479407564e-05


### 실습 02. Regularization

실습영상에서 이번 dataset은 feature selection을 사용한다면 더 좋은 성능을 보일 수 있음을 확인했습니다

두 번째 target에 대해 regularization을 적용하고, regularization term에 따라 모델 성능이 어떻게 변하는지 평가지표로 확인해주세요

In [97]:
from sklearn.linear_model import Lasso, Ridge
from plotly.subplots import make_subplots
from plotly import express as px

rd_coef_list=[]
for lamda in range(0,100):
    rd=Ridge(lamda*0.00000003)
    rd.fit(x_train, y_train)
    rd_coef_list.append(rd.coef_.ravel())

ls_coef_list = []
for lamda in range(1,100):
    ls=Lasso(lamda*0.0001)
    ls.fit(x_train,y_train)
    ls_coef_list.append(ls.coef_.ravel())
    
re_coefs=np.array(rd_coef_list).transpose()
ls_coefs=np.array(ls_coef_list).transpose()

C:\Users\micke\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 2506.7163456352428, tolerance: 60.797674683378744

C:\Users\micke\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 695.9860330419033, tolerance: 60.797674683378744

C:\Users\micke\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 320.4197604622459, tolerance: 60.797674683378744

C:\Users\micke\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 183.49854916351615, t

In [102]:
rd_score_list=[]
for lamda in range(0,100):
    rd=Ridge(lamda*0.000000003)
    rd.fit(x_train,y_train)
    rd_score_list.append([mean_squared_error(y_test, rd.predict(x_test))**0.5,
                          mean_absolute_error(y_test, rd.predict(x_test)),
                          r2_score(y_test, rd.predict(x_test))])
ls_score_list=[]
for lamda in range(1,100):
    ls=Lasso(lamda*0.0001)
    ls.fit(x_train, y_train)
    ls_score_list.append([mean_squared_error(y_test,ls.predict(x_test))**0.5,
                          mean_absolute_error(y_test, ls.predict(x_test)),
                          r2_score(y_test, ls.predict(x_test))])

colors=px.colors.qualitative.Alphabet_r
error_name=['MSE','MAE','R2']
fig = make_subplots(1,3, subplot_titles=error_name,
                   specs=[[{"secondary_y": True},{"secondary_y": True},{"secondary_y": True}]])

for n in range(1,4):
    fig.add_trace(
        go.Scatter(
                y=np.array(ls_score_list)[:,n-1], name=f'Lasso-{error_name[n-1]}'
                ), secondary_y= True,
    row=1,col=n)
    
    fig.add_trace(
        go.Scatter(
                y=np.array(rd_score_list)[:,n-1], name=f'Ridge-{error_name[n-1]}'
                ), secondary_y= True,
    row=1,col=n)

fig_options={'layout':dict(template='simple_white',
                          width=1000, height=400,)}
fig.update(fig_options)
fig.show()

C:\Users\micke\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 2506.7163456352428, tolerance: 60.797674683378744

C:\Users\micke\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 695.9860330419033, tolerance: 60.797674683378744

C:\Users\micke\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 320.4197604622459, tolerance: 60.797674683378744

C:\Users\micke\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 183.49854916351615, t

### 실습 03. Various regression methods

이전까지 실습한 3가지 기법을 사용하여 회귀를 시도해보겠습니다

3가지 회귀를 시도하시고 성능을 어떻게 해야 기법들의 성능이 좋아질지 작성해주세요

모든 기법을 다 사용하셔도 되고, 하나의 기법에 대해서만 고민해보셔도 괜찮습니다

code로 결과를 보이고 아래 설명을 마크다운으로 작성부탁드립니다

아래는 예시이며 다양한 평가지표와 방법들을 생각해보세요

ex) 스케일링한 dataset에 대해 Ridge coeffficients로 변수선택 후, knn으로 적절한 이웃을 찾았더니 $R^2$가 전보다 a만큼 좋아졌다

설명을 작성해주세요